In [1]:
import csv
import numpy as np
import random
import statistics

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 1.1 Load Training Data

In [33]:
def load_train(file_name, torch_tensor = True):
    images = []
    with open(file_name, newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if row[0] == "ID":
                continue
            
            row_data = process_row(row)
            
            images.append(row_data)
            
    if torch_tensor:
        return torch.tensor(images)
    else:
        return np.array(images)    

In [34]:
def process_row(row):
    # map all elements to float, replace NaNs with 0.001
    row = list(map(string_to_float, row))
    
    # get mean
    row_mean = statistics.mean(row)

    # replace 0.001s with mean
    row = [modify_mean(elt, row_mean) for elt in row]    
    return row

def string_to_float(x):
    if x == "NaN":
        return float(0.001)
    else:
        return float(x)
    
def modify_mean(x, mean):
    if x == 0.001:
        return mean
    else:
        return x
    

    
data2 = load_train('additional_training.csv')

In [35]:
data2

tensor([[2.4800e+02, 1.8301e-01, 0.0000e+00,  ..., 2.7834e-02, 1.8301e-01,
         1.0000e+00],
        [2.4900e+02, 0.0000e+00, 0.0000e+00,  ..., 3.7892e-02, 1.7745e-01,
         1.0000e+00],
        [2.5000e+02, 0.0000e+00, 0.0000e+00,  ..., 4.6496e-02, 1.1939e-02,
         0.0000e+00],
        ...,
        [2.4640e+03, 4.3786e-02, 6.8936e-01,  ..., 2.7723e-02, 1.8035e-02,
         1.0000e+00],
        [2.4650e+03, 0.0000e+00, 1.0512e+00,  ..., 2.8936e-02, 4.2728e-02,
         1.0000e+00],
        [2.4660e+03, 0.0000e+00, 0.0000e+00,  ..., 3.4974e-02, 2.4695e-02,
         1.0000e+00]])

In [36]:
data1 = load_train('training.csv')
print("data1: ", data1.shape)

data1_x = data1[:,1:4609]
data1_y = data1[:,4609].long()

print("data1_x: ", data1_x.shape)
print("data1_y: ", data1_y.shape)

data1:  torch.Size([247, 4610])
data1_x:  torch.Size([247, 4608])
data1_y:  torch.Size([247])


### 1.2 Additional Training Data

In [37]:
data2 = load_train('additional_training.csv')

In [38]:
data2_x = data2[:, 1:4609]
data2_y = data2[:,4609].long()

print(data2_x.shape)
print(data2_y.shape)

torch.Size([2219, 4608])
torch.Size([2219])


In [39]:
trn_x = torch.cat((data1_x, data2_x))
trn_y = torch.cat((data1_y, data2_y))

print("trn_x: ", trn_x.shape)
print("trn_y: ", trn_y.shape)

trn_x:  torch.Size([2466, 4608])
trn_y:  torch.Size([2466])


### 1.2 Shuffle Data

In [40]:
def shuffle_data(data_x, data_y):
    num_data = data_y.shape[0]
    
    random_indices = [i for i in range(num_data)]
    random.shuffle(random_indices)
    
    new_data_x = data_x[random_indices, :]
    new_data_y = data_y[random_indices]   
    
    return new_data_x, new_data_y

#trn_x, trn_y = shuffle_data(trn_x, trn_y)

### 2. Model Architecture

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4608, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256,32)
        self.fc4 = nn.Linear(32, 2)
        self.softmax_layer = nn.LogSoftmax(dim=0)

    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x)
        x = F.relu(x)
        
        x = self.fc4(x)
        x = self.softmax_layer(x)
        
        return x

In [56]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4608, 4608)
        self.fc2 = nn.Linear(4608, 1000)
        self.fc3 = nn.Linear(1000, 2)
        self.softmax_layer = nn.LogSoftmax(dim=0)

    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.fc3(x)
        x = self.softmax_layer(x)
        
        return x

In [60]:
model = Net()

### 3. Calculate Accuracy

In [61]:
def calculate_accuracy(model, data_x, data_y):
    
    num_images = data_y.shape[0]

    num_correct = 0
    
    for i in range(num_images):
        output = model(data_x[i,:])
        pred = int(torch.argmax(output).data)    
        #print(output)
        if pred == data_y[i]:
            num_correct += 1
        
    return num_correct / num_images

### 4. Train

In [72]:
EPOCH = 1
LEARNING_RATE = 0.01
MOMENTUM = 0.5
NUM_IMAGES = trn_x.shape[0]

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
#optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCH):

    trn_x, trn_y = shuffle_data(trn_x, trn_y)

    optimizer.zero_grad()    
        
    for i in range(NUM_IMAGES):
        x_i = trn_x[i,:]
        output = model(x_i).view(1,2)
        gold = trn_y[i].view(1)
        
        loss = F.nll_loss(output, gold)

        model.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(epoch)
    print(calculate_accuracy(model, trn_x, trn_y))    
        

0
0.981346309813463


### 5. Predict Test Data

In [27]:
def load_test(torch_tensor = True):
    images = []
    with open('testing.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if row[0] == "ID":
                continue
            row_data = list(map(float, row))
            images.append(row_data)
            
    if torch_tensor:
        return torch.tensor(images)
    else:
        return np.array(images)   

In [28]:
tst_data = load_test()
tst_data.shape

torch.Size([11874, 4609])

In [29]:
tst_x = tst_data[:,1:4609]
tst_x.shape

torch.Size([11874, 4608])

In [75]:
TEST_IMAGES = tst_x.shape[0]
predictions = []


for i in range(TEST_IMAGES):
    output = model(tst_x[i,:])
    pred = int(torch.argmax(output).data)    
    predictions.append(pred)

predictions

[0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,


### 6. Write predictions into csv

In [76]:
def write_results(predictions):
    num_ids = len(predictions)
    

    with open('submission.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "prediction"])
        for i in range(num_ids):
            writer.writerow([i+1, predictions[i]])

    
write_results(predictions)